In [2]:
from drugrelink import pairs


In [3]:
list1,list2,list3,list4 =pairs.test_pairs(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv',train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

In [11]:
len(list1[0][])

209168

In [2]:
!head /home/lxu/drugrelink/data/validation-statuses.tsv

compound_id	compound_name	disease_id	disease_name	n_trials	status_trials	status_drugcentral
DB01048	Abacavir	DOID:10652	Alzheimer's disease	0	0	0
DB01048	Abacavir	DOID:9206	Barrett's esophagus	0	0	0
DB01048	Abacavir	DOID:13241	Behcet's disease	0	0	0
DB01048	Abacavir	DOID:11949	Creutzfeldt-Jakob disease	0	0	0
DB01048	Abacavir	DOID:8778	Crohn's disease	0	0	0
DB01048	Abacavir	DOID:11555	Fuchs' endothelial dystrophy	0	0	0
DB01048	Abacavir	DOID:11119	Gilles de la Tourette syndrome	0	0	0
DB01048	Abacavir	DOID:12361	Graves' disease	0	0	0
DB01048	Abacavir	DOID:2986	IgA glomerulonephritis	0	0	0


In [ ]:
def find_difference(l1,l2):
    pairs = [[],[]]
    for i in range(len(l1[0])):
        for y in range (len(l2[0])):
            if l1[0][i] == l2[0][y] and l1[1][i] != l2[1][y]:
                pairs[0].append(l2[0][y])
                pairs[1].append(l2[1][y])
                
    return pairs
            

positive4L2 = find_difference(list1,list2)
len(positive4L2)

In [ ]:
positive4L3 = find_difference(list1,list3)
print(len(positive4L3))

In [ ]:
positive4L4 = find_difference(list1,list4)
print(len(positive4L4))

In [ ]:
def find_negative(l1,l2):
    pairs =[]
    for i in range(len(l1[0])):
        for y in range(len(l2[0])):
            if l1[1][i] == 1 and 

In [3]:
from drugrelink import pairs
df = pairs.data_non_overlap(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv',train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

In [4]:
print(len(df.loc[df['label']==0]),len(df.loc[df['label']==1]))
print(df.head())
print(df['compound'][1])

202313 6855
   Unnamed: 0           compound              disease  label
0           0  Compound::DB01048  Disease::DOID:10652      0
1           1  Compound::DB05812  Disease::DOID:10652      0
2           2  Compound::DB00659  Disease::DOID:10652      0
3           3  Compound::DB00284  Disease::DOID:10652      0
4           4  Compound::DB01193  Disease::DOID:10652      0
Compound::DB05812


In [6]:
import pickle
from node2vec.edges import HadamardEmbedder
with open('/home/lxu/drugrelink/resources/predictive_model/edge2vec/0/word2vec_model.pickle', 'rb') as file:
    word2vec = pickle.load(file)
def vectors(df):
    vectors=[]
    for _,row in df[['compound','disease']].iterrows():
        c=row['compound']
        
        d=row['disease']
        
   
        edges_embs = HadamardEmbedder(keyed_vectors=word2vec.wv)
        vector = edges_embs[(c,d)]
        vectors.append(vector)
    return vectors

In [7]:
y = df['label']
y_org = y.copy()
X=vectors(df)

In [8]:
from sklearn.linear_model import LogisticRegression
import numpy as np

def retrain(hidden_number):

    y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_number
    )]=0
    
    model = LogisticRegression().fit(X,y)
    return y, model


In [16]:
y,model = retrain(1000)

   
import pandas as pd
results = pd.DataFrame({
    'truth'      : y_org,                    # True labels
    'label'      : y,        # Labels shown to models
    'predict_label': model.predict(X),
    'output_std' : model.predict_proba(X)[:,1]   # Random forest's scores
}, columns = ['truth', 'label', 'predict_label','output_std'])

/home/lxu/anaconda3/envs/my-new-rdkit-env/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/lxu/anaconda3/envs/my-new-rdkit-env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [17]:
c = df['compound']
d = df['disease']
names=c.append(d)
r=names.append(results)

In [18]:
for i,row in r[['truth','label']].iterrows():
    l1=[]
    if row['truth'].values != row['label'].values and row['truth'].values == row['predict_label'].values:
        l1.append(i)
l1

     

KeyError: 'predict_label'

In [23]:
 plabel = model.predict(X) 

In [24]:
plabel[2]

0

In [25]:
from collections import Counter
a = Counter(plabel)

In [26]:
a

Counter({0: 209098, 1: 70})